<a href="https://colab.research.google.com/github/dohyeongkim97/papers/blob/master/gan_augmentator_en_train_de_faire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
drive.mount('/content/drive', force_remount=True)

df = pd.read_csv("./drive/MyDrive/paper_data-master/train.csv")
test = pd.read_csv("./drive/MyDrive/paper_data-master/test.csv")

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
data_path = './drive/MyDrive/paper_data-master/'
df = pd.read_csv(data_path+'train.csv')
test = pd.read_csv(data_path+'test.csv')

In [4]:
def make_dataset(data, tokenizer, device):
    tokenized = tokenizer(
        text = data.text.tolist(),
        padding= 'longest',
        truncation = True,
        return_tensors = 'pt'
    )
    input_ids = tokenized['input_ids'].to(device)
    attention_mask = tokenized['attention_mask'].to(device)
    if 'label' in data.columns:
        labels = torch.tensor(data['label'].values, dtype=torch.long).to(device)
        return TensorDataset(input_ids, attention_mask, labels)
    else:
        return TensorDataset(input_ids, attention_mask)

In [6]:
def get_dataloader(dataset, sampler, batch_size):
    data_sampler = sampler(dataset)
    dataloader = DataLoader(dataset, sampler = data_sampler, batch_size = batch_size)
    return dataloader

In [9]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler
from torch import optim
from transformers import BertForSequenceClassification
from torch import nn
import math
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = BertTokenizer.from_pretrained(
    pretrained_model_name_or_path='bert-base-multilingual-cased',
    do_lower_case = False
)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
df

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1
...,...,...,...,...,...
2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",Congress amended the Clean Air Act through the...,1
2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.","Alliance Bond Fund, Inc., an investment fund, ...",1
2475,TRAIN_2475,Peguero,United States,"In 1992, the District Court sentenced Manuel D...",0
2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per...",0


In [14]:
def frame_make(df):
    if 'first_party_winner' in df.columns:
        data = pd.DataFrame(columns = ['text', 'target'])
        df['first_party_winner'] = df['first_party_winner'].astype(int)
        data['text'] = 'first_party:' + df['first_party'] + 'second_party:' + df['second_party'] + 'facts:' + df['facts'] + '\nwinner: '
        data['target'] = df['first_party_winner']
        return data
    else:
        data = pd.DataFrame(columns = ['text'])
        data['text'] = 'first_party:' + df['first_party'] + 'second_party:' + df['second_party'] + 'facts:' + df['facts'] + '\nwinner: '
        return data

In [15]:
df_data = frame_make(df)
df_data = make_dataset(df_data, tokenizer, device)
df_data = get_dataloader(df_data, RandomSampler, 16)

In [19]:
df_data

In [20]:
df_data = frame_make(df)

In [21]:
df_data

,text,target
0,first_party:Phil A. St. Amantsecond_party:Herm...,1
1,first_party:Stephen Duncansecond_party:Lawrenc...,0
2,first_party:Billy Joe Magwoodsecond_party:Tony...,1
3,first_party:Linklettersecond_party:Walkerfacts...,0
4,first_party:William Earl Fikessecond_party:Ala...,1
...,...,...
2473,"first_party:HollyFrontier Cheyenne Refining, L...",1
2474,"first_party:Grupo Mexicano de Desarrollo, S. A...",1
2475,first_party:Peguerosecond_party:United Statesf...,0
2476,first_party:Immigration and Naturalization Ser...,0


In [18]:
real_data = torch.randint(0, vocab_size, (32, seq_length), dtype=torch.long)
real_data

NameError: name 'vocab_size' is not defined

In [16]:
read_data = np.stack(df['text'].apply(lambda x: np.fromstring(x[1:-1], sep=',', dtype=int)))

KeyError: 'text'

In [5]:
real_data = np.stack(df['tokenized_text'].apply(lambda x: np.fromstring(x[1:-1], sep=',', dtype=int)))

KeyError: 'tokenized_text'